In [ ]:
import matplotlib.pyplot as plt
import time

from geometry import Camera, Scene, Sphere, Plane, Light
from raytracer import RayTracer
from Tone_map import ecualization_inv, gammaFunc


def create_cornell_box():
    """Crear una escena estilo Cornell Box."""
    scene = Scene()
    scene.add_light(Light([0.0, 1.8, -1.0], intensity=1.2))

    scene.add_sphere(Sphere([-0.3, -0.2, -0.8], 0.3, [0.9, 0.9, 0.9], material=[1, 0, 0]))
    scene.add_sphere(Sphere([0.9, -0.3, -1.2], 0.2, [0.8, 0.2, 0.2], material=[0, 1, 0], ior=0.7))
    scene.add_sphere(Sphere([0.0, 0.2, -1.5], 0.4, [0.2, 0.8, 0.5], material=[0, 0, 1], ior=1.5))

    scene.add_plane(Plane([0, -0.5, 0], [0, 1, 0], [0.9, 0.9, 0.9]))     # suelo
    scene.add_plane(Plane([0, 2.0, 0], [0, -1, 0], [0.9, 0.9, 0.9]))     # techo
    scene.add_plane(Plane([-1.5, 0, 0], [1, 0, 0], [0.4, 0.2, 0.2]))     # pared roja
    scene.add_plane(Plane([1.5, 0, 0], [-1, 0, 0], [0.2, 0.8, 0.5]))     # pared verde
    scene.add_plane(Plane([0, 0, -2.5], [0, 0, 1], [0.9, 0.9, 0.9]))     # fondo
    return scene


def configure_camera(width, height):
    """Devuelve una cámara adecuada según la escena."""
    aspect_ratio = width / height
    return Camera(
            fov_deg=60,
            aspect_ratio=aspect_ratio,
            lookfrom=(0, 0.5, 1.5),
            lookat=(0, 0, -1),
            vup=(0, 1, 0)
        )


def display_image(image, title="Ray Tracing", save_path=None):
    """Mostrar la imagen renderizada y opcionalmente guardarla."""
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title(title)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Imagen guardada en: {save_path}")
    plt.show()


def run_render(width, height, 
               use_gpu=True, samples=1, bounces=2, save_path=None):
    """Ejecuta el render de forma interactiva (profiling-ready)."""
    print(f"Resolución: {width}x{height} | {'GPU' if use_gpu else 'CPU'}")

    scene = create_cornell_box()

    camera = configure_camera(width, height)

    tracer = RayTracer(width=width, height=height, prefer_gpu=use_gpu, samples=samples, bounces=bounces)
    tracer.set_camera(camera)

    start = time.time()
    image = tracer.render(scene)
    end = time.time()

    print(f"✅ Render completado en {end - start:.2f} segundos")

    image = gammaFunc(1, 1.2, image)  # Aplicar corrección gamma

    display_image(image, "" , save_path)
    return image, end - start

display_image(run_render(width=1920, height=1080, use_gpu=True, samples=500, bounces=20)[0],
              title="Cornell Box - GPU Render", save_path="cornell_box_gpu.png")


Resolución: 1920x1080 | GPU
Ray Tracer: 1920x1080, usando GPU
Renderizando en GPU (Optimizado): 1920x1080, 100 SPP, 40 rebotes
Error en renderizado: Failed in cuda mode pipeline (step: nopython frontend)
Untyped global name 'local': Cannot determine Numba type of <class 'type'>

File "gpu_renderer.py", line 321:
    def render_kernel_optimized(image, width, height, 
        <source elided>
        # Estado RNG local
        rng_state = local.array(1, types.uint32)
        ^

During: Pass nopython_type_inference
Intentando fallback a CPU...


/home/jorges/Escritorio/Prog/Grafica/cpu_renderer.py:15: RuntimeWarning: invalid value encountered in divide
  return v / fast_norm(v)


KeyboardInterrupt: 